In [1]:
#coding:utf-8 
from bs4 import BeautifulSoup #bs4モジュールからBeautifulSoupクラスをインポートする
import requests
import csv
import scrapelib
from urllib.request import urlopen

In [2]:
from ipywidgets import FloatProgress
from IPython.display import display
from time import sleep

In [ ]:
#全体をタグに関してまわす？
#スパースになりすぎる？
###男女で分けたほうがいいかも...男にスカート推薦してもな
with open('brands.csv', 'wt',encoding='utf8') as fbrand:
    with open('tags.csv','wt',encoding='utf8') as ftag:
        with open('imgs.csv','wt',encoding='utf8') as fimg:
            fp = FloatProgress(min=0, max=10398956-8604629+1)
            display(fp)
            writer_brand = csv.writer(fbrand, lineterminator='\n')
            writer_tag = csv.writer(ftag, lineterminator='\n')
            writer_img = csv.writer(fimg, lineterminator='\n')
            s = scrapelib.Scraper(requests_per_minute=80) #１分間に何回リクエストを投げるか
            brand_list_list=[]
            tag_list_list=[]
            others_list_list=[]
            count=0 #データ数のカウント用
            for number in range(8604629,10398956):#コーディネートの指定10万件___ここ変えるならfp.valueも変える
            #for number in range(9004626,9562847):#コーディネートの指定
                #----------------------------例外処理---------------------------------------------#
                try: 
                    t=s.get('http://wear.jp/erierieri/'+str(number)+'/')
                except: # 全て
                    continue;
                #--------------------------------------------------------------------------------#
                soup2=BeautifulSoup(t.text,"lxml")
                #print(number) #今見てるページ
                fp.value+=1
                #-----------------------------カウント用のファイルを作る-------------------------------------#
                if(fp.value % 10000==0):
                    with open('./data/process'+str(fp.value)+'.csv','wt',encoding='utf8') as fcount:
                        print(fp.value)
                    #fcount.close()
                #--------------------------------------------------------------------------------#
                #-------------------------------#404NotFoundの検出#------------------------------#
                if(soup2.find(id='tag')==None): 
                    continue
                if(soup2.find(id='item')==None): 
                    continue
                if(soup2.find(id='coordinate_img')==None):
                    continue
                #--------------------------------------------------------------------------------#
                brand_list=[]
                tag_list=[]
                prof_img_list=[]
                #------------------------------#つかえるかもしれない#----------------------------#
                #----------------#print(soup2.find_all(class_='link_list'))#---------------------#

                #soup2.find(id='tag').select('a')
                #------------------------------#ブランド情報の取得#------------------------------#
                #print(soup2.find(id='item').select('.brand'))
                for a in soup2.find(id='item').select('.brand > a'):
                    brand_list.append(a.string)
                #--------------------------------------------------------------------------------#
                #----------------------------------#タグ情報の取得#------------------------------#
                for a in soup2.find(id='tag').select('a'):
                    tag_list.append(a.string)
                #--------------------------------------------------------------------------------#
                #--------------------------------#モデル情報の取得#------------------------------#
                for a in soup2.select('.model_info > a'):
                    attributes = [x.strip() for x in a.text.split('/')]
                    #print(attributes)
                #for attribute in attributes:
                    #prof_img_list.append(attribute)
                    prof_img_list.append(attributes[0])
                    prof_img_list.append(attributes[1])
                    prof_img_list.append(attributes[2])
                #--------------------------------------------------------------------------------#
                #----------------------------#コーディネート画像の取得#--------------------------#
                img=soup2.find(id='coordinate_img').find(class_='img').find('img')
                #print(img['src'])
                img_url=img['src']
                prof_img_list.append(img_url)
                #--------------------------------------------------------------------------------#
                #-------------------------------#各csvへの書き込み#------------------------------#
                if len(brand_list)>=2 and len(tag_list)>=1and len(prof_img_list)==4:#3,4とかにするべき？
                    writer_tag.writerow(tag_list)
                    writer_brand.writerow(brand_list)
                    writer_img.writerow(prof_img_list)
                    brand_list_list.append(brand_list)
                    tag_list_list.append(tag_list)
                    others_list_list.append(prof_img_list)
#                     print(brand_list)
#                     print(tag_list)
#                     print(prof_img_list)
                    count+=1;
    
                if(count % 10000==0): #1万回ごとにファイルを積み重ねていく
                    with open('./data/process/brand/'+str(count)+'.csv','wt',encoding='utf8') as fprocess_brand:
                        writer_process_brand = csv.writer(fprocess_brand, lineterminator='\n')
                        for bl in brand_list_list:
                            writer_process_brand.writerow(bl)
                    with open('./data/process/tag/'+str(count)+'.csv','wt',encoding='utf8') as fprocess_tag:
                        writer_process_tag = csv.writer(fprocess_tag, lineterminator='\n')
                        for t in tag_list_list:
                            writer_process_tag.writerow(t)
                    with open('./data/process/others/'+str(count)+'.csv','wt',encoding='utf8') as fprocess_others:
                        writer_process_others = csv.writer(fprocess_others, lineterminator='\n')
                        for o in others_list_list:
                            writer_process_others.writerow(o)                      
                #--------------------------------------------------------------------------------#
        #fimg.close()
    #ftag.close()
#fbrand.close()

print("データ数: "+str(count))

with open('end.csv', 'wt',encoding='utf8') as fend:
    writer_end = csv.writer(fend, lineterminator='\n')
    writer_end.writerow(end)
fend.close()

The installed widget Javascript is the wrong version.
